In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [26]:
# Define state

class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float  # Strike Rate
    bpb: float  # Balls Per Boundary
    boundary_percent: float  # Boundary Percentage
    summary: str


In [37]:
def calculate_sr(state: BatsmanState):
    if state['balls'] == 0:
        return 0.0
    sr = (state['runs'] / state['balls']) * 100
    #state['sr'] = sr
    return {'sr': sr}

In [54]:
def calculate_bpb(state: BatsmanState):
    bpb = state['balls']/(state['fours'] + state['sixes'])
    # state['bpb'] = bpb
    return {'bpb': bpb}

In [55]:
def calculate_boundry_percent(state: BatsmanState):
    if state['runs'] == 0:
        state['boundary_percent'] = 0.0
    else:
        boundary_percent = ((state['fours'] * 4 + state['sixes'] * 6) / state['runs']) * 100
        return {'boundary_percent': boundary_percent}


In [62]:
def summary(state: BatsmanState):
    summary = f"""
    Strike Rate - {state['sr']} \n
    Balls Per Boundary - {state['bpb']} \n
    Boundary Percent - {state['boundary_percent']} \n
    """
    # state['summary'] = summary
    return {'summary': summary}
   

In [63]:
# Define the state graph

graph = StateGraph(BatsmanState)

#add node
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percent', calculate_boundry_percent)
graph.add_node('summary', summary)

# edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percent', 'summary')

graph.add_edge('summary', END)

workflow = graph.compile()

In [64]:
initial_state = {
    'runs': 100,
    'balls': 50,
    'fours': 6,
    'sixes': 4 
}

workflow.invoke(initial_state)

{'runs': 100,
 'balls': 50,
 'fours': 6,
 'sixes': 4,
 'sr': 200.0,
 'bpb': 5.0,
 'boundary_percent': 48.0,
 'summary': '\n    Strike Rate - 200.0 \n\n    Balls Per Boundary - 5.0 \n\n    Boundary Percent - 48.0 \n\n    '}